In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash import Dash
from dash import dash
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "accuser"
shelter = AnimalShelter(username, password)


df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'],inplace=True)

# Define color coding for outcome types
outcome_colors = {
    "Adoption": "green",
    "Transfer": "blue",
    "Return to Owner": "purple",
    "Euthanasia": "red",
    "Died": "black",
    "Missing": "orange",
}

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.H1('Austin Animal Center Data/Map', style={'color': '#2c3e50'})),
    html.Center(html.P('By: Darrell Walker', style={'color': '#2c3e50'})),
    html.Hr(),
    
    # Filter controls
    html.Div([
        html.Label("Filter by Animal Type:", style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='filter-animal-type',
            options=[{'label': i, 'value': i} for i in df['animal_type'].unique()],
            placeholder="Select animal type...",
            multi=True
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        html.Label("Filter by Outcome Type:", style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='filter-outcome-type',
            options=[{'label': i, 'value': i} for i in df['outcome_type'].unique()],
            placeholder="Select outcome type...",
            multi=True
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        html.Label("Search by Breed:", style={'font-weight': 'bold'}),
        dcc.Input(id="search-breed", type="text", placeholder="Type breed name..."),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    html.Hr(),
    
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        selected_rows=[0],
        row_selectable="single",
        page_size=15,
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': '#34495e', 'color': 'white'},
        style_cell={'textAlign': 'center', 'padding': '5px'},
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': '#f2f2f2'}
        ]
    ),
    html.Br(),
    html.Hr(),
    
    # Selected Animal Name (Above Map)
    html.Div(id='selected-animal-name', style={
        'textAlign': 'center',
        'fontWeight': 'bold',
        'fontSize': '20px',
        'marginBottom': '10px'  # Add spacing before the map
    }),

    # Interactive Map (Below the Name)
    html.Div(id='map-id', style={'display': 'flex', 'justify-content': 'center', 'align-items': 'center', 'height': '600px'}),
    html.Hr(),
    
     # Outcome Pie Chart
    dcc.Graph(id='outcome-pie-chart'),

])
#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        # If no columns are selected, return an empty list (no styles)
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Table Filtering Callback
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-animal-type', 'value'),
     Input('filter-outcome-type', 'value'),
     Input('search-breed', 'value')]
)
def update_table(animal_type, outcome_type, breed_search):
    dff = df.copy()

    if animal_type:
        dff = dff[dff['animal_type'].isin(animal_type)]
    
    if outcome_type:
        dff = dff[dff['outcome_type'].isin(outcome_type)]

    if breed_search:
        dff = dff[dff['breed'].str.contains(breed_search, case=False, na=False)]

    return dff.to_dict('records')


# Outcome Type Pie Chart
@app.callback(
    Output('outcome-pie-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(filtered_data):
    dff = pd.DataFrame(filtered_data)
    fig = px.pie(dff, names='outcome_type', title="Animal Outcome Distribution", color='outcome_type',
                 color_discrete_map=outcome_colors)
    return fig


# Map Update Callback
# Update Map and Animal Name Callback
@app.callback(
    [Output('map-id', "children"),
     Output('selected-animal-name', "children")],  # Two outputs (map & name)
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map_and_name(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)

    # Default text if no row is selected
    selected_animal_name = "No animal selected"

    # Default map
    default_map = dl.Map(style={'width': '1000px', 'height': '600px'}, center=[30.75, -97.48], zoom=10)

    # Check if any row is selected
    if not selected_rows or len(dff) == 0 or selected_rows[0] >= len(dff):
        return (default_map, selected_animal_name)  # Return as tuple

    # Select the row data
    selected_animal = dff.iloc[selected_rows[0]]

    # Update the name display
    if pd.notnull(selected_animal["name"]):
        selected_animal_name = f"Name: {selected_animal['name']}"
    elif selected_animal['name'] == "":
        selected_animal_name = "No Name"
    else:
        selected_animal_name = "Name: Unknown"

    # Ensure the selected row has valid latitude and longitude
    if pd.isnull(selected_animal["location_lat"]) or pd.isnull(selected_animal["location_long"]):
        return (default_map, selected_animal_name)

    # Create a single marker for the selected item
    marker = dl.Marker(
        position=[selected_animal["location_lat"], selected_animal["location_long"]],
        children=[
            dl.Popup([
                html.H3(f"Animal: {selected_animal['animal_type']}"),
                html.P(f"Breed: {selected_animal['breed']}"),
                html.P(f"Outcome: {selected_animal['outcome_type']}"),
                html.P(f"Age: {selected_animal['age_upon_outcome']}"),
            ])
        ]
    )

    # Create updated map
    updated_map = dl.Map(
        style={'width': '1000px', 'height': '600px'},
        center=[selected_animal["location_lat"], selected_animal["location_long"]], zoom=12,
        children=[
            dl.TileLayer(),
            marker  # Only one marker appears at a time
        ]
    )

    return (updated_map, selected_animal_name)  # Returning a tuple


# Run App
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:13433/
